In [20]:
3.2 (a)
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math as math


U, X = np.linalg.qr(np.random.randn(80,80))
V, X = np.linalg.qr(np.random.randn(80,80))

result = []
for i in range(1,81):
    result.append(2**(-i))

S = np.diag(result)
A = np.dot(U, S, V)

def clgs(A):
    m = A.shape[0]
    n = A.shape[1]
    Q = np.eye(m,n)
    V = A.copy()
    R = np.zeros((n,n))
    for j in range(1,n+1):
        for i in range(1,j):
            R[i-1,j-1] = np.dot(np.transpose(Q[:,i-1]),A[:,j-1])
            V[:,j-1] = V[:,j-1]-np.dot(R[i-1,j-1],Q[:,i-1])

        R[j-1,j-1] = np.linalg.norm(V[:,j-1])
        Q[:,j-1] = V[:,j-1]/R[j-1,j-1]
    return Q, R

def mgs(A):
    m = A.shape[0]
    n = A.shape[1]
    Q = A.copy()
    R = np.zeros((n,n))
    for i in range(1,n):
        R[i-1,i-1] = np.linalg.norm(Q[:,i-1])
        Q[:,i-1] = Q[:,i-1]/R[i-1,i-1]
        R[i-1,i:] = np.dot(np.transpose(Q[:,i-1]),Q[:,i:])
        Q[:,i:] = Q[:,i:]-np.dot(Q[i:,i-1],R[i-1,i:])

    R[n-1,n-1] = np.linalg.norm(Q[:,n-1])
    Q[:,n-1] = Q[:,n-1]/R[n-1,n-1]
    return Q,R

Qc, Rc = clgs(A)
Qm, Rm = mgs(A)



-24.082399653118497
-24.082399653775312


In [16]:
3.2(b)
A=np.array([[.70000, .70711], [.70001, .70711]])
Qm, Rm = mgs(A)

def qrhouseholder(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy() 

    for j in range(n):

        x = R[j:, j]
        normx = np.linalg.norm(x)
        rho = -np.sign(x[0])
        u1 = x[0] - rho * normx
        u = x / u1
        u[0] = 1
        beta = -rho * u1 / normx

        R[j:, :] = R[j:, :] - beta * np.outer(u, u).dot(R[j:, :])
        Q[:, j:] = Q[:, j:] - beta * Q[:, j:].dot(np.outer(u, u))
        
    return Q, R

Q, R = qrhouseholder(A)

a=np.linalg.norm((np.dot(np.transpose(Q),Q))-np.eye(2))
b=np.linalg.norm((np.dot(np.transpose(Qm),Qm))-np.eye(2))

print(a)
print(b)//householder is more stable

2.3551386880256624e-16
1.414213562337019


In [24]:
3.3

def givensrotation(a,b):
    if b == 0:
        c = 1
        s = 0
    else:
        if abs(b) > abs(a):
            r = a / b
            s = 1 / math.sqrt(1 + r**2)
            c = s*r
        else:
            r = b / a;
            c = 1 / math.sqrt(1 + r**2);
            s = c*r
    return c,s


def qrgivens(A):
    m,n = A.shape
    Q = np.eye(m)
    R = np.copy(A)
    for j in range(1,n+1):
        for i in range(m,j,-1):
            G = np.eye(m);
            c,s = givensrotation( R[i-2,j-1],R[i-1,j-1] );
            G[i-2,i-2] = c
            G[i-2,i-1] = -s
            G[i-1,i-2] = s
            G[i-1,i-1] = c
            R = np.dot(np.transpose(G),R);
            Q = np.dot(Q,G);
    return Q,R


Q1, R1 = np.linalg.qr(A)
Q2 , R2 = qrgivens(A)

print(Q1)
print(Q2)//the results are same


[[-0.11698151  0.1156284   0.08886704 ... -0.00665939 -0.13394269
  -0.08491115]
 [ 0.06325439 -0.08519754 -0.12115256 ...  0.18890599 -0.12262938
   0.10830266]
 [-0.12135456 -0.05574644 -0.25693199 ... -0.02644169  0.11305239
   0.08356987]
 ...
 [-0.00089703 -0.11575184 -0.05655247 ...  0.15968908  0.02730006
  -0.05761688]
 [-0.08258977  0.12925612 -0.1851872  ... -0.04561434 -0.18066349
  -0.02204637]
 [ 0.09003068  0.07230008 -0.07547884 ... -0.0501314   0.04664116
   0.17393087]]
[[-0.11698151 -0.1156284  -0.08886704 ...  0.00665939  0.13394269
  -0.08491115]
 [ 0.06325439  0.08519754  0.12115256 ... -0.18890599  0.12262938
   0.10830266]
 [-0.12135456  0.05574644  0.25693199 ...  0.02644169 -0.11305239
   0.08356987]
 ...
 [-0.00089703  0.11575184  0.05655247 ... -0.15968908 -0.02730006
  -0.05761688]
 [-0.08258977 -0.12925612  0.1851872  ...  0.04561434  0.18066349
  -0.02204637]
 [ 0.09003068 -0.07230008  0.07547884 ...  0.0501314  -0.04664116
   0.17393087]]
